In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from matplotlib.collections import LineCollection
from numpy.linalg import cholesky # Needed for SDE simulation noise generation

# --- Explicit Eigendecomposition Provided by User ---
# Eigenvalues (Lambda)
Lambda_provided = np.array([
    [-0.1 - 1j, 0],
    [0, -0.1 + 1j]
], dtype=complex)

# Eigenvectors (S)
S_provided = np.array([
    [1 - 1j, 1 + 1j],
    [1, 1]
], dtype=complex)

# Inverse of Eigenvectors (S_inv)
S_inv_provided = 0.5 * np.array([
    [1j, 1 - 1j],
    [-1j, 1 + 1j]
], dtype=complex)

# --- System Parameters ---
# Continuous-Time System Matrix (A = S @ Lambda @ S_inv)
A = np.real(S_provided @ Lambda_provided @ S_inv_provided)
# Note: This results in A = [[-0.1, -1.0], [1.0, -0.1]], a circular spiral.

# Process Noise Std Dev (sigma_process) from simulation
SIGMA_PROCESS = 0.1
# Correlation between x1 and x2 process noise components
CORRELATION_Q = 0.5
# Continuous-Time Process Noise Covariance (Q = G*G^T).
Q_variance = SIGMA_PROCESS**2
Q = np.array([
    [Q_variance, CORRELATION_Q * Q_variance],
    [CORRELATION_Q * Q_variance, Q_variance]
])
# Cholesky decomposition for process noise (L*L^T = Q)
L = cholesky(Q)

# Measurement Matrix (C = H in simulation)
C = np.identity(2)

# Measurement Noise Std Dev (sigma_measurement) from simulation
SIGMA_MEASUREMENT = 0.5
# Correlation between x1 and x2 measurement noise components
CORRELATION_R = 0.3
# Measurement Noise Covariance (R) - Now non-diagonal
R_variance = SIGMA_MEASUREMENT**2
R = np.array([
    [R_variance, CORRELATION_R * R_variance],
    [CORRELATION_R * R_variance, R_variance]
])
# Cholesky decomposition for measurement noise (M*M^T = R)
M = cholesky(R)

# --- Visualization Parameters ---
FIXED_END_TIME = 15.0 # Total simulation time (T)
INITIAL_STATE = np.array([5.0, 0.0]) # Starting point of the spiral at t=0
DT = 0.01 # Time step for Euler-Maruyama simulation
N_POINTS_MEASUREMENT = 200 # Number of noisy measurements to plot (subsampled)


# --- Analytical DLE Solution (Kept for completeness but not used in this plot) ---

# Pre-calculate Eigenbasis components for faster DLE solution
Lambda = np.diag(Lambda_provided)
S = S_provided
S_inv = S_inv_provided
S_dagger = S.conj().T
S_inv_dagger = S_inv.conj().T
Lambda_Q = np.diag(S_inv @ Q @ S_inv_dagger)
Lambda_C = np.diag(S_inv @ C @ S)
Lambda_R = np.diag(S_inv @ R @ S_inv_dagger)


def scalar_dle_solution_phi(lambda_k, lambda_Q_k, dt):
    """
    Computes the scalar solution phi(lambda, lambda_Q, dt) for the DLE.
    (Not used in this simulation plot, but retained from previous versions)
    """
    real_lambda = np.real(lambda_k)

    if np.isclose(real_lambda, 0.0):
        return lambda_Q_k * dt
    else:
        exponential_term = np.exp(2 * real_lambda * dt)
        denominator = -2 * real_lambda
        return lambda_Q_k * (1 - exponential_term) / denominator


def compute_propagated_covariance(dt):
    """
    Computes the total propagated measurement covariance V_C(dt).
    (Not used in this simulation plot, but retained from previous versions)
    """
    d = A.shape[0]
    Lambda_V_diag = np.zeros(d, dtype=complex)
    for k in range(d):
        Lambda_V_diag[k] = scalar_dle_solution_phi(Lambda[k], Lambda_Q[k], dt)

    abs_Lambda_C_sq = np.abs(Lambda_C)**2
    term1 = abs_Lambda_C_sq * Lambda_V_diag
    real_Lambda = np.real(Lambda)
    exp_term = np.exp(2 * real_Lambda * dt)
    term2 = exp_term * Lambda_R
    Lambda_V_C_diag = term1 + term2
    V_C = S @ np.diag(Lambda_V_C_diag) @ S_dagger
    return np.real(V_C)


# --- New Simulation and Plotting Function ---

def simulate_and_plot_sde(T, dt):
    """
    Simulates the true state (X) and noisy measurements (Y) of the 2D SDE.
    """
    N_steps = int(T / dt)
    time_steps = np.arange(0, T, dt)

    X = np.zeros((N_steps, 2)) # True state
    Y = np.zeros((N_steps, 2)) # Measured state

    X[0] = INITIAL_STATE

    # Pre-calculate factor for process noise term
    L_sqrt_dt = L @ (np.sqrt(dt) * np.identity(2))

    # Euler-Maruyama Simulation Loop
    for k in range(N_steps - 1):
        # 1. Process Noise (dW_t ~ N(0, dt*Q))
        Z_k = np.random.randn(2)
        dW_term = L_sqrt_dt @ Z_k

        # 2. True State Update: X_k+1 = X_k + A * X_k * dt + dW_term
        drift_term = (A @ X[k]) * dt
        X[k+1] = X[k] + drift_term + dW_term

        # 3. Measurement Noise (v_t ~ N(0, R))
        V_k = M @ np.random.randn(2)

        # 4. Measured State: Y_k = C * X_k + v_t
        Y[k] = C @ X[k] + V_k # Measure the state before the update (or use X[k+1], small difference)

    # Final measurement at the last time step
    Y[-1] = C @ X[-1] + M @ np.random.randn(2)

    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(10, 10))

    # 1. Plot the True State (Smooth Spiral)
    ax.plot(X[:, 0], X[:, 1], color='black', linestyle='-', linewidth=1.5, alpha=0.9,
            label='True State Trajectory', zorder=2)

    # 2. Plot the Measured State (Noisy Observations)
    # Subsample the measurements to prevent the plot from being too dense
    subsample_indices = np.linspace(0, N_steps - 1, N_POINTS_MEASUREMENT, dtype=int)
    Y_subsampled = Y[subsample_indices]

    ax.scatter(Y_subsampled[:, 0], Y_subsampled[:, 1],
               color='red', marker='.', s=15, alpha=0.6,
               label='Noisy Measurements', zorder=1)

    # 3. Mark Start/End Points
    ax.plot(X[0, 0], X[0, 1], 'o', color='green', markersize=8,
            label='Start (t=0)', zorder=3)
    ax.plot(X[-1, 0], X[-1, 1], 'D', color='blue', markersize=8,
            label=f'End (t={T}s)', zorder=3)

    # 4. Plot Formatting
    ax.set_title(f'SDE Simulation: True State (Black) vs. Noisy Measurements (Red)', fontsize=16)
    ax.set_xlabel(r'State $x_1$', fontsize=12)
    ax.set_ylabel(r'State $x_2$', fontsize=12)
    ax.set_aspect('equal', adjustable='box')
    ax.grid(True, linestyle=':', alpha=0.5)
    ax.legend(loc='upper right')
    plt.tight_layout()
    plt.show()


# --- Run Calculation and Plot (Replaced plot_spiral_uncertainty call) ---
print("--- Running SDE Simulation Plot ---")
print(f"Simulating SDE up to T = {FIXED_END_TIME} s with dt = {DT} s.")

simulate_and_plot_sde(FIXED_END_TIME, DT)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from numpy.linalg import cholesky

# --- Helper function for drawing confidence ellipses ---
def plot_ellipse(ax, mean, cov, color, label_suffix, sigma=2, linestyle='--', linewidth=2):
    """
    Plots the confidence ellipse for a given covariance matrix.
    """
    # Calculate eigenvalues and eigenvectors
    # D: eigenvalues (variances), V: eigenvectors (principal axes)
    D, V = np.linalg.eig(cov)

    # Ensure real, non-negative eigenvalues for the 2D case
    D = np.maximum(np.real(D), 1e-9)

    # Calculate the angle of rotation for the largest eigenvector
    angle = np.degrees(np.arctan2(V[1, 0], V[0, 0]))

    # Calculate width and height of the ellipse (sigma * sqrt(eigenvalue))
    width, height = sigma * 2 * np.sqrt(D)

    # Create the ellipse object
    from matplotlib.patches import Ellipse
    ellipse = Ellipse(xy=mean, width=width, height=height, angle=angle,
                      edgecolor=color, facecolor='none', linestyle=linestyle, linewidth=linewidth, zorder=4)

    # Add the ellipse to the axis
    ax.add_patch(ellipse)

    # Add a dummy artist for the legend entry if this is the first one
    if label_suffix:
        ax.plot([], [], color=color, linestyle=linestyle, linewidth=linewidth,
                label=label_suffix)


# --- Explicit Eigendecomposition Provided by User ---
# Eigenvalues (Lambda)
Lambda_provided = np.array([
    [-0.1 - 1j, 0],
    [0, -0.1 + 1j]
], dtype=complex)

# Eigenvectors (S)
S_provided = np.array([
    [1 - 1j, 1 + 1j],
    [1, 1]
], dtype=complex)

# Inverse of Eigenvectors (S_inv)
S_inv_provided = 0.5 * np.array([
    [1j, 1 - 1j],
    [-1j, 1 + 1j]
], dtype=complex)

# --- System Parameters ---
# Continuous-Time System Matrix (A = S @ Lambda @ S_inv)
A = np.real(S_provided @ Lambda_provided @ S_inv_provided)

# Process Noise Std Dev (sigma_process) from simulation
SIGMA_PROCESS = 0.1
# Correlation between x1 and x2 process noise components
CORRELATION_Q = 0.5
# Continuous-Time Process Noise Covariance (Q = G*G^T).
Q_variance = SIGMA_PROCESS**2
Q = np.array([
    [Q_variance, CORRELATION_Q * Q_variance],
    [CORRELATION_Q * Q_variance, Q_variance]
])
# Cholesky decomposition for process noise (L*L^T = Q)
L = cholesky(Q)

# Measurement Matrix (C = H in simulation)
C = np.identity(2)

# Measurement Noise Std Dev (sigma_measurement) from simulation
SIGMA_MEASUREMENT = 0.5
# Correlation between x1 and x2 measurement noise components
CORRELATION_R = 0.3
# Measurement Noise Covariance (R) - Now non-diagonal
R_variance = SIGMA_MEASUREMENT**2
R = np.array([
    [R_variance, CORRELATION_R * R_variance],
    [CORRELATION_R * R_variance, R_variance]
])
# Cholesky decomposition for measurement noise (M*M^T = R)
M = cholesky(R)

# --- Visualization Parameters ---
FIXED_END_TIME = 15.0 # Total simulation time (T)
INITIAL_STATE = np.array([5.0, 0.0]) # Starting point of the spiral at t=0
DT = 0.01 # Time step for Euler-Maruyama simulation
N_POINTS_MEASUREMENT = 200 # Number of noisy measurements to plot (subsampled)

# --- New Smoothing Visualization Parameters (UPDATED) ---
# Added t=1.0 to the list of target times
TARGET_TIMES = [1.0, 5.0, 10.0, 13.0, 15.0] # The 5 fixed times to map all measurements to
# Added 'green' for the new target time at 1.0
TARGET_COLORS = ['green', 'blue', 'red', 'orange', 'purple'] # Colors for the 5 resulting clouds

# --- Analytical DLE Solution ---

# Pre-calculate Eigenbasis components for faster DLE solution
Lambda = np.diag(Lambda_provided)
S = S_provided
S_inv = S_inv_provided
S_dagger = S.conj().T
S_inv_dagger = S_inv.conj().T
# Q transformed into the eigenbasis
Lambda_Q = np.diag(S_inv @ Q @ S_inv_dagger)
# C transformed into the eigenbasis
Lambda_C = np.diag(S_inv @ C @ S)
# R transformed into the eigenbasis
Lambda_R = np.diag(S_inv @ R @ S_inv_dagger)


def scalar_dle_solution_phi(lambda_k, lambda_Q_k, dt):
    """
    Computes the scalar solution phi(lambda, lambda_Q, dt) for the DLE.
    This gives the steady-state variance of the transformed system.
    """
    real_lambda = np.real(lambda_k)

    if np.isclose(real_lambda, 0.0):
        # Solution for Re(lambda) = 0 (Purely oscillatory)
        return lambda_Q_k * dt
    else:
        # Solution for Re(lambda) != 0
        exponential_term = np.exp(2 * real_lambda * dt)
        denominator = -2 * real_lambda
        # This is the integral of exp(2 * A * t) * Q dt, which solves the DLE for V_C.
        return lambda_Q_k * (1 - exponential_term) / denominator


def compute_propagated_covariance(dt):
    """
    Computes the total propagated measurement covariance V_C(dt) = V_P(dt) + Phi(dt) * R * Phi(dt)^T.
    V_P(dt) is the noise accumulated from Q.
    This is calculated using the eigenbasis solution to the DLE.
    """
    d = A.shape[0]
    # V_P = S @ diag(Lambda_V_diag) @ S_dagger
    Lambda_V_diag = np.zeros(d, dtype=complex)
    for k in range(d):
        Lambda_V_diag[k] = scalar_dle_solution_phi(Lambda[k], Lambda_Q[k], dt)

    # Full covariance V_C = V_P + V_R
    abs_Lambda_C_sq = np.abs(Lambda_C)**2
    # V_P term (accumulated process noise)
    term1 = abs_Lambda_C_sq * Lambda_V_diag
    # V_R term (propagated measurement noise: Phi * R * Phi^T)
    real_Lambda = np.real(Lambda)
    exp_term = np.exp(2 * real_Lambda * dt)
    term2 = exp_term * Lambda_R

    # Total V_C in the eigenbasis: diag(Lambda_V_C_diag)
    Lambda_V_C_diag = term1 + term2

    # Transform back to original basis: V_C = S @ diag(Lambda_V_C_diag) @ S_dagger
    V_C = S @ np.diag(Lambda_V_C_diag) @ S_dagger
    return np.real(V_C)


# --- Simulation Function ---
def simulate_sde(T, dt):
    """
    Simulates the true state (X) and noisy measurements (Y) of the 2D SDE.
    Returns the time steps, true states, and measurements.
    """
    N_steps = int(T / dt)
    time_steps = np.arange(0, T, dt)

    X = np.zeros((N_steps, 2)) # True state
    Y = np.zeros((N_steps, 2)) # Measured state

    X[0] = INITIAL_STATE

    # Pre-calculate factor for process noise term
    L_sqrt_dt = L @ (np.sqrt(dt) * np.identity(2))

    # Euler-Maruyama Simulation Loop
    for k in range(N_steps - 1):
        # 1. Process Noise (dW_t ~ N(0, dt*Q))
        Z_k = np.random.randn(2)
        dW_term = L_sqrt_dt @ Z_k

        # 2. True State Update: X_k+1 = X_k + A * X_k * dt + dW_term
        drift_term = (A @ X[k]) * dt
        X[k+1] = X[k] + drift_term + dW_term

        # 3. Measurement Noise (v_t ~ N(0, R))
        V_k = M @ np.random.randn(2)

        # 4. Measured State: Y_k = C * X_k + v_t
        Y[k] = C @ X[k] + V_k

    # Final measurement at the last time step
    Y[-1] = C @ X[-1] + M @ np.random.randn(2)

    return time_steps, X, Y

# --- Mapping and Plotting Function ---
def plot_mapped_measurements(time_steps, X_true, Y_measurements, target_times, target_colors):
    """
    Maps all measurements (Y) forward/backward to the specified target times
    using the matrix exponential, and overlays the optimal smoothed covariance.
    """
    fig, ax = plt.subplots(figsize=(10, 10))

    # 1. Plot the True State Trajectory
    ax.plot(X_true[:, 0], X_true[:, 1], color='black', linestyle='-', linewidth=1.5, alpha=0.9,
            label='True Trajectory', zorder=1)

    # Flag to ensure the smoothed ellipse legend is only added once
    smoothed_legend_added = False

    # 2. Map and Plot Measurement Clouds and Optimal Smoothed Ellipses
    for t_target, color in zip(target_times, target_colors):

        # Find the true state at the target time for plotting the center
        # np.argmin finds the index closest to t_target
        target_index = np.argmin(np.abs(time_steps - t_target))
        X_target = X_true[target_index]

        # Initialize total precision (P_total = sum(P_C) for all k)
        # The optimal smoothed covariance V_smoothed is the inverse of the total precision P_total
        P_total = np.zeros((2, 2))
        mapped_cloud = []

        # --- Map Measurements (Forecast/Backcast) & Accumulate Precision ---
        for t_k, Y_k in zip(time_steps, Y_measurements):
            Delta_t = t_target - t_k

            # 1. Propagate Measurement: Y_mapped = Phi(Delta_t) * Y_k
            Phi = expm(A * Delta_t)
            Y_mapped = Phi @ Y_k
            mapped_cloud.append(Y_mapped)

            # 2. Calculate V_C(Delta_t) and P_C(Delta_t)
            # V_C is the covariance of the mapped measurement (including both process and measurement noise effects)
            V_C = compute_propagated_covariance(Delta_t)
            try:
                # Accumulate precision: P_total += V_C_inv
                P_total += np.linalg.inv(V_C)
            except np.linalg.LinAlgError:
                print(f"Warning: Singular V_C encountered at t_k={t_k:.2f} to t_target={t_target:.2f}")

        mapped_cloud = np.array(mapped_cloud)

        # 3. Calculate Smoothed Covariance V_smoothed = P_total_inv
        try:
            V_smoothed = np.linalg.inv(P_total)
        except np.linalg.LinAlgError:
            print(f"Error: Total precision P_total is singular at t_target={t_target}")
            continue # Skip plotting this ellipse

        # Plot the cloud of mapped measurements
        ax.scatter(mapped_cloud[:, 0], mapped_cloud[:, 1],
                   color=color, marker='.', s=10, alpha=0.3,
                   label=f'Mapped Cloud ($t = {t_target}$)', zorder=2)

        # Plot the True State at the target time
        ax.plot(X_target[0], X_target[1], 'X', color='black', markeredgecolor=color, markersize=12,
                label=f'True $X({t_target})$', zorder=5)

        # --- Plot Analytic Smoothed Ellipse ---
        # Plot the 2-sigma ellipse of the optimally smoothed covariance V_smoothed
        plot_ellipse(ax, X_target, V_smoothed, 'black',
                     label_suffix=r'Analytic $\mathbf{V}_{\text{smoothed}}$ ($\sigma=2$)' if not smoothed_legend_added else None,
                     linestyle='-', linewidth=3) # Use solid, thick black line for the final result

        # Only add the ellipse label once for the legend
        smoothed_legend_added = True

    # 3. Mark Start Point
    ax.plot(X_true[0, 0], X_true[0, 1], 'o', color='green', markersize=8,
            label='Start (t=0)', zorder=3)

    # 4. Plot Formatting
    # ax.set_title(r'Optimal Smoothed Covariance $\mathbf{V}_{\text{smoothed}}$ from All Measurements', fontsize=16)
    ax.set_xlabel(r'State $x_1$', fontsize=12)
    ax.set_ylabel(r'State $x_2$', fontsize=12)
    ax.set_aspect('equal', adjustable='box')
    plt.xlim(X_true[:, 0].min() - 5, X_true[:, 0].max() + 5)
    ax.grid(True, linestyle=':', alpha=0.5)

    # Handle legend with duplicate labels
    handles, labels = ax.get_legend_handles_labels()
    unique = []
    seen = set()
    for h, l in zip(handles, labels):
        if l not in seen:
            unique.append((h, l))
            seen.add(l)
    # ax.legend(*zip(*unique), loc='lower right')

    plt.tight_layout()
    plt.show()


# --- Run Simulation and Plot ---
print("--- Running SDE Measurement Mapping Plot with 5 Target Times ---")
time_steps, X_true, Y_measurements = simulate_sde(FIXED_END_TIME, DT)

# Call the function with the updated lists
plot_mapped_measurements(time_steps, X_true, Y_measurements, TARGET_TIMES, TARGET_COLORS)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from numpy.linalg import cholesky

# --- Helper function for drawing confidence ellipses ---
def plot_ellipse(ax, mean, cov, color, label_suffix, sigma=2, linestyle='--', linewidth=2):
    """
    Plots the confidence ellipse for a given covariance matrix.
    """
    # Calculate eigenvalues and eigenvectors
    # D: eigenvalues (variances), V: eigenvectors (principal axes)
    D, V = np.linalg.eig(cov)

    # Ensure real, non-negative eigenvalues for the 2D case
    D = np.maximum(np.real(D), 1e-9)

    # Calculate the angle of rotation for the largest eigenvector
    angle = np.degrees(np.arctan2(V[1, 0], V[0, 0]))

    # Calculate width and height of the ellipse (sigma * sqrt(eigenvalue))
    width, height = sigma * 2 * np.sqrt(D)

    # Create the ellipse object
    from matplotlib.patches import Ellipse
    ellipse = Ellipse(xy=mean, width=width, height=height, angle=angle,
                      edgecolor=color, facecolor='none', linestyle=linestyle, linewidth=linewidth, zorder=4)

    # Add the ellipse to the axis
    ax.add_patch(ellipse)

    # Add a dummy artist for the legend entry if this is the first one
    if label_suffix:
        ax.plot([], [], color=color, linestyle=linestyle, linewidth=linewidth,
                label=label_suffix)


# --- Explicit Eigendecomposition Provided by User ---
# Eigenvalues (Lambda)
Lambda_provided = np.array([
    [-0.1 - 1j, 0],
    [0, -0.1 + 1j]
], dtype=complex)

# Eigenvectors (S)
S_provided = np.array([
    [1 - 1j, 1 + 1j],
    [1, 1]
], dtype=complex)

# Inverse of Eigenvectors (S_inv)
S_inv_provided = 0.5 * np.array([
    [1j, 1 - 1j],
    [-1j, 1 + 1j]
], dtype=complex)

# --- System Parameters ---
# Continuous-Time System Matrix (A = S @ Lambda @ S_inv)
A = np.real(S_provided @ Lambda_provided @ S_inv_provided)

# Process Noise Std Dev (sigma_process) from simulation
SIGMA_PROCESS = 0.2
# Correlation between x1 and x2 process noise components
CORRELATION_Q = 0.4
# Continuous-Time Process Noise Covariance (Q = G*G^T).
Q_variance = SIGMA_PROCESS**2
Q = np.array([
    [Q_variance, CORRELATION_Q * Q_variance],
    [CORRELATION_Q * Q_variance, Q_variance]
])
# Cholesky decomposition for process noise (L*L^T = Q)
L = cholesky(Q)

# Measurement Matrix (C = H in simulation)
C = np.identity(2)

# Measurement Noise Std Dev (sigma_measurement) from simulation
SIGMA_MEASUREMENT = 0.5
# Correlation between x1 and x2 measurement noise components
CORRELATION_R = 0.3
# Measurement Noise Covariance (R) - Now non-diagonal
R_variance = SIGMA_MEASUREMENT**2
R = np.array([
    [R_variance, CORRELATION_R * R_variance],
    [CORRELATION_R * R_variance, R_variance]
])
# Cholesky decomposition for measurement noise (M*M^T = R)
M = cholesky(R)

# --- Visualization Parameters ---
FIXED_END_TIME = 15.0 # Total simulation time (T)
INITIAL_STATE = np.array([5.0, 0.0]) # Starting point of the spiral at t=0
DT = 0.01 # Time step for Euler-Maruyama simulation
N_POINTS_MEASUREMENT = 200 # Number of noisy measurements to plot (subsampled)

# --- Smoothing Visualization Parameters ---
# Added t=1.0 to the list of target times
TARGET_TIMES = [1.0, 5.0, 10.0, 13.0, 15.0] # The 5 fixed times to map all measurements to
# Added 'green' for the new target time at 1.0
TARGET_COLORS = ['green', 'blue', 'red', 'orange', 'purple'] # Colors for the 5 resulting clouds

# --- Analytical DLE Solution ---

# Pre-calculate Eigenbasis components for faster DLE solution
Lambda = np.diag(Lambda_provided)
S = S_provided
S_inv = S_inv_provided
S_dagger = S.conj().T
S_inv_dagger = S_inv.conj().T
# Q transformed into the eigenbasis
Lambda_Q = np.diag(S_inv @ Q @ S_inv_dagger)
# C transformed into the eigenbasis
Lambda_C = np.diag(S_inv @ C @ S)
# R transformed into the eigenbasis
Lambda_R = np.diag(S_inv @ R @ S_inv_dagger)


def scalar_dle_solution_phi(lambda_k, lambda_Q_k, dt):
    """
    Computes the scalar solution phi(lambda, lambda_Q, dt) for the DLE.
    This gives the steady-state variance of the transformed system.
    """
    real_lambda = np.real(lambda_k)

    if np.isclose(real_lambda, 0.0):
        # Solution for Re(lambda) = 0 (Purely oscillatory)
        return lambda_Q_k * dt
    else:
        # Solution for Re(lambda) != 0
        exponential_term = np.exp(2 * real_lambda * dt)
        denominator = -2 * real_lambda
        # This is the integral of exp(2 * A * t) * Q dt, which solves the DLE for V_C.
        return lambda_Q_k * (1 - exponential_term) / denominator


def compute_propagated_covariance(dt):
    """
    Computes the total propagated measurement covariance V_C(dt) = V_P(dt) + Phi(dt) * R * Phi(dt)^T.
    V_P(dt) is the noise accumulated from Q.
    This is calculated using the eigenbasis solution to the DLE.
    """
    d = A.shape[0]
    # V_P = S @ diag(Lambda_V_diag) @ S_dagger
    Lambda_V_diag = np.zeros(d, dtype=complex)
    for k in range(d):
        Lambda_V_diag[k] = scalar_dle_solution_phi(Lambda[k], Lambda_Q[k], dt)

    # Full covariance V_C = V_P + V_R
    abs_Lambda_C_sq = np.abs(Lambda_C)**2
    # V_P term (accumulated process noise)
    term1 = abs_Lambda_C_sq * Lambda_V_diag
    # V_R term (propagated measurement noise: Phi * R * Phi^T)
    real_Lambda = np.real(Lambda)
    exp_term = np.exp(2 * real_Lambda * dt)
    term2 = exp_term * Lambda_R

    # Total V_C in the eigenbasis: diag(Lambda_V_C_diag)
    Lambda_V_C_diag = term1 + term2

    # Transform back to original basis: V_C = S @ np.diag(Lambda_V_C_diag) @ S_dagger
    V_C = S @ np.diag(Lambda_V_C_diag) @ S_dagger
    return np.real(V_C)


# --- Simulation Function ---
def simulate_sde(T, dt):
    """
    Simulates the true state (X) and noisy measurements (Y) of the 2D SDE.
    Returns the time steps, true states, and measurements.
    """
    N_steps = int(T / dt)
    time_steps = np.arange(0, T, dt)

    X = np.zeros((N_steps, 2)) # True state
    Y = np.zeros((N_steps, 2)) # Measured state

    X[0] = INITIAL_STATE

    # Pre-calculate factor for process noise term
    L_sqrt_dt = L @ (np.sqrt(dt) * np.identity(2))

    # Euler-Maruyama Simulation Loop
    for k in range(N_steps - 1):
        # 1. Process Noise (dW_t ~ N(0, dt*Q))
        Z_k = np.random.randn(2)
        dW_term = L_sqrt_dt @ Z_k

        # 2. True State Update: X_k+1 = X_k + A * X_k * dt + dW_term
        drift_term = (A @ X[k]) * dt
        X[k+1] = X[k] + drift_term + dW_term

        # 3. Measurement Noise (v_t ~ N(0, R))
        V_k = M @ np.random.randn(2)

        # 4. Measured State: Y_k = C * X_k + v_t
        Y[k] = C @ X[k] + V_k

    # Final measurement at the last time step
    V_k = M @ np.random.randn(2)
    Y[-1] = C @ X[-1] + V_k

    return time_steps, X, Y

# --- Mapping and Plotting Function (MODIFIED FOR CAUSALITY) ---
def plot_mapped_measurements(time_steps, X_true, Y_measurements, target_times, target_colors):
    """
    Maps all CAUSAL measurements (Y, where t_k <= t_target) forward/backward
    to the specified target times, and overlays the optimal causal filter covariance.

    The causal filter covariance V_filter is the inverse of the total precision
    P_total = sum(V_C(t_target - t_k)^-1) for all t_k <= t_target.
    """
    fig, ax = plt.subplots(figsize=(10, 10))

    # 1. Plot the True State Trajectory
    ax.plot(X_true[:, 0], X_true[:, 1], color='black', linestyle='-', linewidth=1.5, alpha=0.9,
            label='True Trajectory', zorder=1)

    # Flag to ensure the filter ellipse legend is only added once
    filter_legend_added = False

    # 2. Map and Plot Measurement Clouds and Optimal Causal Filter Ellipses
    for t_target, color in zip(target_times, target_colors):

        # Find the true state at the target time for plotting the center
        # np.argmin finds the index closest to t_target
        target_index = np.argmin(np.abs(time_steps - t_target))
        X_target = X_true[target_index]

        # Initialize total precision (P_total = sum(P_C) for all k)
        # The optimal causal filter covariance V_filter is the inverse of the total precision P_total
        P_total = np.zeros((2, 2))
        mapped_cloud = []

        # --- Map Measurements (Forecast/Backcast) & Accumulate Precision ---
        for t_k, Y_k in zip(time_steps, Y_measurements):
            Delta_t = t_target - t_k

            # *** CAUSALITY CHECK: Only use measurements taken BEFORE or AT the target time ***
            # Floating point comparisons require a small tolerance
            if Delta_t < -1e-9: # If t_k > t_target, then Delta_t is negative.
                continue

            # 1. Propagate Measurement: Y_mapped = Phi(Delta_t) * Y_k
            Phi = expm(A * Delta_t)
            Y_mapped = Phi @ Y_k
            mapped_cloud.append(Y_mapped)

            # 2. Calculate V_C(Delta_t) and P_C(Delta_t)
            # V_C is the covariance of the mapped measurement (including both process and measurement noise effects)
            V_C = compute_propagated_covariance(Delta_t)
            try:
                # Accumulate precision: P_total += V_C_inv
                P_total += np.linalg.inv(V_C)
            except np.linalg.LinAlgError:
                print(f"Warning: Singular V_C encountered at t_k={t_k:.2f} to t_target={t_target:.2f}")

        mapped_cloud = np.array(mapped_cloud)

        # 3. Calculate Causal Filter Covariance V_filter = P_total_inv
        try:
            V_filter = np.linalg.inv(P_total)
        except np.linalg.LinAlgError:
            print(f"Error: Total precision P_total is singular at t_target={t_target}")
            continue # Skip plotting this ellipse

        # Plot the cloud of mapped measurements (subsample for clarity if there are too many points)
        # Only plot every 50th point for a cleaner cloud visualization
        subsample_rate = max(1, len(mapped_cloud) // 200)
        ax.scatter(mapped_cloud[::subsample_rate, 0], mapped_cloud[::subsample_rate, 1],
                   color=color, marker='.', s=10, alpha=0.3,
                   label=f'Mapped Causal Cloud ($t = {t_target}$)', zorder=2)

        # Plot the True State at the target time
        ax.plot(X_target[0], X_target[1], 'X', color='black', markeredgecolor=color, markersize=12,
                label=f'True $X({t_target})$', zorder=5)

        # --- Plot Analytic Causal Filter Ellipse ---
        # Plot the 2-sigma ellipse of the optimally filtered covariance V_filter
        # plot_ellipse(ax, X_target, V_filter, 'black',
        #              label_suffix=r'Analytic $\mathbf{V}_{\text{causal filter}}$ ($\sigma=2$)' if not filter_legend_added else None,
        #              linestyle='-', linewidth=3) # Use solid, thick black line for the final result

        # Only add the ellipse label once for the legend
        filter_legend_added = True

    # 3. Mark Start Point
    ax.plot(X_true[0, 0], X_true[0, 1], 'o', color='green', markersize=8,
            label='Start (t=0)', zorder=3)

    # 4. Plot Formatting
    # ax.set_title(r'Optimal Causal Filter Covariance $\mathbf{V}_{\text{filter}}$ from Prior Measurements', fontsize=16)
    ax.set_xlabel(r'$x$', fontsize=12)
    ax.set_ylabel(r'$y$', fontsize=12)
    ax.set_aspect('equal', adjustable='box')
    plt.xlim(-7,9)
    plt.ylim(-8,8)
    ax.grid(True, linestyle=':', alpha=0.5)
    ax.set_aspect('equal', 'box')

    # Handle legend with duplicate labels
    handles, labels = ax.get_legend_handles_labels()
    unique = []
    seen = set()
    for h, l in zip(handles, labels):
        if l not in seen:
            unique.append((h, l))
            seen.add(l)
    # ax.legend(*zip(*unique), loc='lower right')

    plt.tight_layout()
    plt.show()


# --- Run Simulation and Plot ---
print("--- Running SDE Causal Filter Plot with 5 Target Times ---")
time_steps, X_true, Y_measurements = simulate_sde(FIXED_END_TIME, DT)

# Call the function with the updated lists
plot_mapped_measurements(time_steps, X_true, Y_measurements, TARGET_TIMES, TARGET_COLORS)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm, eig
from matplotlib.collections import LineCollection
from numpy.linalg import cholesky # Needed for SDE simulation noise generation

# --- Helper function for drawing confidence ellipses (copied from previous version) ---
def plot_ellipse(ax, mean, cov, color, label_suffix, sigma=2, linestyle='-', linewidth=2):
    """
    Plots the confidence ellipse for a given covariance matrix.
    """
    # Calculate eigenvalues and eigenvectors
    # D: eigenvalues (variances), V: eigenvectors (principal axes)
    D, V = np.linalg.eig(cov)

    # Ensure real, non-negative eigenvalues for the 2D case
    D = np.maximum(np.real(D), 1e-9)

    # Calculate the angle of rotation for the largest eigenvector
    angle = np.degrees(np.arctan2(V[1, 0], V[0, 0]))

    # Calculate width and height of the ellipse (sigma * sqrt(eigenvalue))
    width, height = sigma * 2 * np.sqrt(D)

    # Create the ellipse object
    from matplotlib.patches import Ellipse
    ellipse = Ellipse(xy=mean, width=width, height=height, angle=angle,
                      edgecolor=color, facecolor='none', linestyle=linestyle, linewidth=linewidth, zorder=4)

    # Add the ellipse to the axis
    ax.add_patch(ellipse)

    # Add a dummy artist for the legend entry if this is the first one
    if label_suffix:
        ax.plot([], [], color=color, linestyle=linestyle, linewidth=linewidth,
                label=label_suffix)


# --- Explicit Eigendecomposition Provided by User ---
# Eigenvalues (Lambda)
Lambda_provided = np.array([
    [-0.1 - 1j, 0],
    [0, -0.1 + 1j]
], dtype=complex)

# Eigenvectors (S)
S_provided = np.array([
    [1 - 1j, 1 + 1j],
    [1, 1]
], dtype=complex)

# Inverse of Eigenvectors (S_inv)
S_inv_provided = 0.5 * np.array([
    [1j, 1 - 1j],
    [-1j, 1 + 1j]
], dtype=complex)

# --- System Parameters ---
# Continuous-Time System Matrix (A = S @ Lambda @ S_inv)
A = np.real(S_provided @ Lambda_provided @ S_inv_provided)
# Note: This results in A = [[-0.1, -1.0], [1.0, -0.1]], a circular spiral.

# Process Noise Std Dev (sigma_process) from simulation
SIGMA_PROCESS = 0.2
# Correlation between x1 and x2 process noise components
CORRELATION_Q = 0.4
# Continuous-Time Process Noise Covariance (Q = G*G^T).
Q_variance = SIGMA_PROCESS**2
Q = np.array([
    [Q_variance, CORRELATION_Q * Q_variance],
    [CORRELATION_Q * Q_variance, Q_variance]
])
# Cholesky decomposition for process noise (L*L^T = Q)
L = cholesky(Q)

# Measurement Matrix (C = H in simulation)
C = np.identity(2)

# Measurement Noise Std Dev (sigma_measurement) from simulation
SIGMA_MEASUREMENT = 0.5
# Correlation between x1 and x2 measurement noise components
CORRELATION_R = 0.3
# Measurement Noise Covariance (R) - Now non-diagonal
R_variance = SIGMA_MEASUREMENT**2
R = np.array([
    [R_variance, CORRELATION_R * R_variance],
    [CORRELATION_R * R_variance, R_variance]
])
# Cholesky decomposition for measurement noise (M*M^T = R)
M = cholesky(R)

# --- Visualization Parameters ---
FIXED_END_TIME = 15.0 # Total simulation time (T)
INITIAL_STATE = np.array([5.0, 0.0]) # Starting point of the spiral at t=0
DT = 0.01 # Time step for Euler-Maruyama simulation
N_POINTS_MEASUREMENT = 200 # Number of noisy measurements to plot (subsampled)

# --- New Smoothing Visualization Parameters ---
TARGET_TIMES = [1.0, 5.0, 10.0, 13.0, 15.0] # The 5 fixed times to map all measurements to
TARGET_COLORS = ['green', 'blue', 'red', 'orange', 'purple'] # Colors for the 5 resulting clouds
SMOOTHING_SUBSAMPLE_RATE = 25 # Use 1 out of every 25 measurements for smoothing and plotting

# --- Analytical DLE Solution (Kept for completeness) ---

# Pre-calculate Eigenbasis components for faster DLE solution
Lambda = np.diag(Lambda_provided)
S = S_provided
S_inv = S_inv_provided
S_dagger = S.conj().T
S_inv_dagger = S_inv.conj().T
Lambda_Q = np.diag(S_inv @ Q @ S_inv_dagger)
Lambda_C = np.diag(S_inv @ C @ S)
Lambda_R = np.diag(S_inv @ R @ S_inv_dagger)


def scalar_dle_solution_phi(lambda_k, lambda_Q_k, dt):
    """
    Computes the scalar solution phi(lambda, lambda_Q, dt) for the DLE.
    """
    real_lambda = np.real(lambda_k)

    if np.isclose(real_lambda, 0.0):
        # Case for pure imaginary or zero eigenvalues (unstable/constant)
        return lambda_Q_k * dt
    else:
        # Standard stable/unstable case
        exponential_term = np.exp(2 * real_lambda * dt)
        denominator = -2 * real_lambda
        return lambda_Q_k * (1 - exponential_term) / denominator


def compute_propagated_covariance(dt):
    """
    Computes the total propagated measurement covariance V_C(dt).
    V_C(dt) = Phi(dt) * Q_c(dt) * Phi(dt)^T + R_c(dt)
    Where Q_c(dt) is the equivalent discrete-time noise covariance and
    R_c(dt) is the equivalent discrete-time measurement noise covariance.
    The analytical solution simplifies this in the eigenbasis.
    """
    d = A.shape[0]
    Lambda_V_diag = np.zeros(d, dtype=complex)
    for k in range(d):
        Lambda_V_diag[k] = scalar_dle_solution_phi(Lambda[k], Lambda_Q[k], dt)

    abs_Lambda_C_sq = np.abs(Lambda_C)**2
    term1 = abs_Lambda_C_sq * Lambda_V_diag
    real_Lambda = np.real(Lambda)
    exp_term = np.exp(2 * real_Lambda * dt)
    term2 = exp_term * Lambda_R
    Lambda_V_C_diag = term1 + term2
    V_C = S @ np.diag(Lambda_V_C_diag) @ S_dagger
    return np.real(V_C)


# --- Simulation Function (Updated to return data) ---

def simulate_sde(T, dt):
    """
    Simulates the true state (X) and noisy measurements (Y) of the 2D SDE.
    Returns the time steps, true states, and measurements.
    """
    N_steps = int(T / dt)
    time_steps = np.arange(0, T, dt)

    X = np.zeros((N_steps, 2)) # True state
    Y = np.zeros((N_steps, 2)) # Measured state

    X[0] = INITIAL_STATE

    # Pre-calculate factor for process noise term
    L_sqrt_dt = L @ (np.sqrt(dt) * np.identity(2))

    # Euler-Maruyama Simulation Loop
    for k in range(N_steps - 1):
        # 1. Process Noise (dW_t ~ N(0, dt*Q))
        Z_k = np.random.randn(2)
        dW_term = L_sqrt_dt @ Z_k

        # 2. True State Update: X_k+1 = X_k + A * X_k * dt + dW_term
        drift_term = (A @ X[k]) * dt
        X[k+1] = X[k] + drift_term + dW_term

        # 3. Measurement Noise (v_t ~ N(0, R))
        V_k = M @ np.random.randn(2)

        # 4. Measured State: Y_k = C * X_k + v_t
        Y[k] = C @ X[k] + V_k

    # Final measurement at the last time step
    Y[-1] = C @ X[-1] + M @ np.random.randn(2)

    return time_steps, X, Y

# --- Updated Mapping and Plotting Function for Causal Filter ---

def plot_mapped_measurements(time_steps, X_true, Y_measurements, target_times, target_colors):
    """
    Maps only past and current measurements (Y_k for t_k <= t_target) forward
    to the specified target times (t_target) using the matrix exponential,
    and overlays the optimal causal filter covariance.
    """
    fig, ax = plt.subplots(figsize=(10, 10))

    # 1. Plot the True State Trajectory
    ax.plot(X_true[:, 0], X_true[:, 1], color='black', linestyle='-', linewidth=1.5, alpha=0.9,
            label='True Trajectory', zorder=1)

    # Flag to ensure the filter ellipse legend is only added once
    filter_legend_added = False

    # 2. Map and Plot Measurement Clouds and Optimal Filter Ellipses
    for t_target, color in zip(target_times, target_colors):

        # Find the true state at the target time for plotting the center
        target_index = np.argmin(np.abs(time_steps - t_target))
        X_target = X_true[target_index]

        # Initialize total precision (P_total = sum(P_C) for all k <= t_target)
        P_total = np.zeros((2, 2))
        mapped_cloud = []

        # --- Map Measurements (Forecast only for t_k <= t_target) & Accumulate Precision ---
        # Note: We use enumerate to apply the subsampling rate for precision and plotting
        for k, (t_k, Y_k) in enumerate(zip(time_steps, Y_measurements)):

            # --- CAUSALITY CONSTRAINT ---
            # Skip any measurement taken after the target time (t_k > t_target)
            if t_k > t_target:
                continue
            # --------------------------

            Delta_t = t_target - t_k # Always positive or zero

            # 1. Propagate Measurement (Forecast to t_target)
            Phi = expm(A * Delta_t)
            Y_mapped = Phi @ Y_k

            # Decide whether to use this point for precision/plotting based on subsampling
            if k % SMOOTHING_SUBSAMPLE_RATE == 0:
                mapped_cloud.append(Y_mapped) # Only add subsampled points to the cloud

                # 2. Calculate V_C(Delta_t) and P_C(Delta_t)
                V_C = compute_propagated_covariance(Delta_t)
                try:
                    # Add precision to total accumulator: P_total += V_C_inv
                    P_total += np.linalg.inv(V_C)
                except np.linalg.LinAlgError:
                    print(f"Warning: Singular V_C encountered at t_k={t_k:.2f}")

        mapped_cloud = np.array(mapped_cloud)

        # 3. Calculate Filtered Covariance V_filter = P_total_inv
        try:
            V_filter = np.linalg.inv(P_total)
        except np.linalg.LinAlgError:
            print(f"Error: Total precision P_total is singular at t_target={t_target}")
            continue # Skip plotting this ellipse

        # Plot the cloud (only subsampled points are included now)
        ax.scatter(mapped_cloud[:, 0], mapped_cloud[:, 1],
                   color=color, marker='.', s=20, alpha=0.5,
                   label=f'Mapped Cloud ($t = {t_target}$)', zorder=2)

        # --- Plot Analytic Filter Ellipse ---
        # The filter covariance (V_filter) should be equal to the P-matrix in a standard Kalman Filter
        # that has been run using all measurements up to t_target.
        plot_ellipse(ax, X_target, V_filter, 'black',
                     label_suffix=r'Analytic $\mathbf{V}_{\text{filter}}$' if not filter_legend_added else None,
                     linestyle='-', linewidth=3, sigma=2) # sigma=2 (2-sigma ellipse)

        # Only add the ellipse label once for the legend
        filter_legend_added = True

    # 3. Mark Start Point
    ax.plot(X_true[0, 0], X_true[0, 1], 'o', color='green', markersize=8,
            label='Start (t=0)', zorder=3)

    # 4. Plot Formatting
    # ax.set_title(r'Optimal Causal Filter Covariance $\mathbf{V}_{\text{filter}}$ from Past Measurements', fontsize=16)
    ax.set_xlabel(r'State $x_1$', fontsize=12)
    ax.set_ylabel(r'State $x_2$', fontsize=12)
    ax.set_aspect('equal', adjustable='box')
    plt.xlim(-7,9)
    plt.ylim(-8,8)
    ax.grid(True, linestyle=':', alpha=0.5)

    # Handle legend with duplicate labels
    handles, labels = ax.get_legend_handles_labels()
    unique = []
    seen = set()
    for h, l in zip(handles, labels):
        if l not in seen:
            unique.append((h, l))
            seen.add(l)
    # ax.legend(*zip(*unique), loc='lower right')

    plt.tight_layout()
    plt.show()


# --- Run Simulation and Plot ---
print("--- Running SDE Causal Filter Plot ---")
time_steps, X_true, Y_measurements = simulate_sde(FIXED_END_TIME, DT)

# The plot function is now modified to enforce causality
plot_mapped_measurements(time_steps, X_true, Y_measurements, TARGET_TIMES, TARGET_COLORS)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm, eig
from matplotlib.collections import LineCollection
from numpy.linalg import cholesky # Needed for SDE simulation noise generation

# --- Helper function for drawing confidence ellipses (copied from previous version) ---
def plot_ellipse(ax, mean, cov, color, label_suffix, sigma=2, linestyle='-', linewidth=2):
    """
    Plots the confidence ellipse for a given covariance matrix.
    """
    # Calculate eigenvalues and eigenvectors
    # D: eigenvalues (variances), V: eigenvectors (principal axes)
    D, V = np.linalg.eig(cov)

    # Ensure real, non-negative eigenvalues for the 2D case
    D = np.maximum(np.real(D), 1e-9)

    # Calculate the angle of rotation for the largest eigenvector
    angle = np.degrees(np.arctan2(V[1, 0], V[0, 0]))

    # Calculate width and height of the ellipse (sigma * sqrt(eigenvalue))
    width, height = sigma * 2 * np.sqrt(D)

    # Create the ellipse object
    from matplotlib.patches import Ellipse
    ellipse = Ellipse(xy=mean, width=width, height=height, angle=angle,
                      edgecolor=color, facecolor='none', linestyle=linestyle, linewidth=linewidth, zorder=4)

    # Add the ellipse to the axis
    ax.add_patch(ellipse)

    # Add a dummy artist for the legend entry if this is the first one
    if label_suffix:
        ax.plot([], [], color=color, linestyle=linestyle, linewidth=linewidth,
                label=label_suffix)


# --- Explicit Eigendecomposition Provided by User ---
# Eigenvalues (Lambda)
Lambda_provided = np.array([
    [-0.1 - 1j, 0],
    [0, -0.1 + 1j]
], dtype=complex)

# Eigenvectors (S)
S_provided = np.array([
    [1 - 1j, 1 + 1j],
    [1, 1]
], dtype=complex)

# Inverse of Eigenvectors (S_inv)
S_inv_provided = 0.5 * np.array([
    [1j, 1 - 1j],
    [-1j, 1 + 1j]
], dtype=complex)

# --- System Parameters ---
# Continuous-Time System Matrix (A = S @ Lambda @ S_inv)
A = np.real(S_provided @ Lambda_provided @ S_inv_provided)
# Note: This results in A = [[-0.1, -1.0], [1.0, -0.1]], a circular spiral.

# Process Noise Std Dev (sigma_process) from simulation
SIGMA_PROCESS = 0.2
# Correlation between x1 and x2 process noise components
CORRELATION_Q = 0.4
# Continuous-Time Process Noise Covariance (Q = G*G^T).
Q_variance = SIGMA_PROCESS**2
Q = np.array([
    [Q_variance, CORRELATION_Q * Q_variance],
    [CORRELATION_Q * Q_variance, Q_variance]
])
# Cholesky decomposition for process noise (L*L^T = Q)
L = cholesky(Q)

# Measurement Matrix (C = H in simulation)
C = np.identity(2)

# Measurement Noise Std Dev (sigma_measurement) from simulation
SIGMA_MEASUREMENT = 0.5
# Correlation between x1 and x2 measurement noise components
CORRELATION_R = 0.3
# Measurement Noise Covariance (R) - Now non-diagonal
R_variance = SIGMA_MEASUREMENT**2
R = np.array([
    [R_variance, CORRELATION_R * R_variance],
    [CORRELATION_R * R_variance, R_variance]
])
# Cholesky decomposition for measurement noise (M*M^T = R)
M = cholesky(R)

# --- Visualization Parameters ---
FIXED_END_TIME = 15.0 # Total simulation time (T)
INITIAL_STATE = np.array([5.0, 0.0]) # Starting point of the spiral at t=0
DT = 0.01 # Time step for Euler-Maruyama simulation
N_POINTS_MEASUREMENT = 200 # Number of noisy measurements to plot (subsampled)

# --- New Smoothing Visualization Parameters ---
TARGET_TIMES = [1.0, 5.0, 10.0, 13.0, 15.0] # The 5 fixed times to map all measurements to
TARGET_COLORS = ['green', 'blue', 'red', 'orange', 'purple'] # Colors for the 5 resulting clouds
SMOOTHING_SUBSAMPLE_RATE = 25 # Use 1 out of every 25 measurements for smoothing and plotting

# --- Analytical DLE Solution (Kept for completeness) ---

# Pre-calculate Eigenbasis components for faster DLE solution
Lambda = np.diag(Lambda_provided)
S = S_provided
S_inv = S_inv_provided
S_dagger = S.conj().T
S_inv_dagger = S_inv.conj().T
Lambda_Q = np.diag(S_inv @ Q @ S_inv_dagger)
Lambda_C = np.diag(S_inv @ C @ S)
Lambda_R = np.diag(S_inv @ R @ S_inv_dagger)


def scalar_dle_solution_phi(lambda_k, lambda_Q_k, dt):
    """
    Computes the scalar solution phi(lambda, lambda_Q, dt) for the DLE.
    This function handles both positive (forecast) and negative (backcast) dt.
    """
    real_lambda = np.real(lambda_k)

    if np.isclose(real_lambda, 0.0):
        # Case for pure imaginary or zero eigenvalues (unstable/constant)
        # Note: This is an approximation for very small real parts
        return lambda_Q_k * dt
    else:
        # Standard stable/unstable case
        exponential_term = np.exp(2 * real_lambda * dt)
        denominator = -2 * real_lambda
        # The equation for the integral of the noise contribution $\int_0^{\Delta t} \mathbf{\Phi}(\Delta t, \tau) \mathbf{Q} \mathbf{\Phi}^T(\Delta t, \tau) d\tau$
        # is $\mathbf{S} \text{diag}(\frac{1 - e^{2\lambda_k \Delta t}}{-2\lambda_k}) \mathbf{S}^\dagger$, which is what this scalar form represents.
        return lambda_Q_k * (1 - exponential_term) / denominator


def compute_propagated_covariance(dt):
    """
    Computes the total propagated measurement covariance V_C(dt).
    V_C(dt) = Phi(dt) * Q_c(dt) * Phi(dt)^T + R_c(dt)
    Where Q_c(dt) is the equivalent discrete-time noise covariance and
    R_c(dt) is the equivalent discrete-time measurement noise covariance.
    The analytical solution simplifies this in the eigenbasis.
    This function is used for both positive (forecast) and negative (backcast) dt.
    """
    d = A.shape[0]
    Lambda_V_diag = np.zeros(d, dtype=complex)
    for k in range(d):
        Lambda_V_diag[k] = scalar_dle_solution_phi(Lambda[k], Lambda_Q[k], dt)

    abs_Lambda_C_sq = np.abs(Lambda_C)**2
    term1 = abs_Lambda_C_sq * Lambda_V_diag
    real_Lambda = np.real(Lambda)
    exp_term = np.exp(2 * real_Lambda * dt)
    term2 = exp_term * Lambda_R
    Lambda_V_C_diag = term1 + term2
    V_C = S @ np.diag(Lambda_V_C_diag) @ S_dagger
    return np.real(V_C)


# --- Simulation Function (Updated to return data) ---

def simulate_sde(T, dt):
    """
    Simulates the true state (X) and noisy measurements (Y) of the 2D SDE.
    Returns the time steps, true states, and measurements.
    """
    N_steps = int(T / dt)
    time_steps = np.arange(0, T, dt)

    X = np.zeros((N_steps, 2)) # True state
    Y = np.zeros((N_steps, 2)) # Measured state

    X[0] = INITIAL_STATE

    # Pre-calculate factor for process noise term
    L_sqrt_dt = L @ (np.sqrt(dt) * np.identity(2))

    # Euler-Maruyama Simulation Loop
    for k in range(N_steps - 1):
        # 1. Process Noise (dW_t ~ N(0, dt*Q))
        Z_k = np.random.randn(2)
        dW_term = L_sqrt_dt @ Z_k

        # 2. True State Update: X_k+1 = X_k + A * X_k * dt + dW_term
        drift_term = (A @ X[k]) * dt
        X[k+1] = X[k] + drift_term + dW_term

        # 3. Measurement Noise (v_t ~ N(0, R))
        V_k = M @ np.random.randn(2)

        # 4. Measured State: Y_k = C * X_k + v_t
        Y[k] = C @ X[k] + V_k

    # Final measurement at the last time step
    Y[-1] = C @ X[-1] + M @ np.random.randn(2)

    return time_steps, X, Y

# --- MODIFIED: Smoother (Non-Causal Filter) Mapping and Plotting Function ---

def plot_smoothed_measurements(time_steps, X_true, Y_measurements, target_times, target_colors):
    """
    Maps ALL measurements (Y_k for all t_k) to the specified target times (t_target)
    using the matrix exponential (Forecast for past, Backcast for future),
    and overlays the optimal NON-CAUSAL SMOOTHER covariance.
    """
    fig, ax = plt.subplots(figsize=(10, 10))

    # 1. Plot the True State Trajectory
    ax.plot(X_true[:, 0], X_true[:, 1], color='black', linestyle='-', linewidth=1.5, alpha=0.9,
            label='True Trajectory', zorder=1)

    # Flag to ensure the smoother ellipse legend is only added once
    smoother_legend_added = False

    # 2. Map and Plot Measurement Clouds and Optimal Smoother Ellipses
    for t_target, color in zip(target_times, target_colors):

        # Find the true state at the target time for plotting the center
        target_index = np.argmin(np.abs(time_steps - t_target))
        X_target = X_true[target_index]

        # Initialize total precision (P_total = sum(P_C) for all k)
        P_total = np.zeros((2, 2))
        mapped_cloud = []

        # --- Map Measurements (Forecast/Backcast) & Accumulate Precision ---
        # Note: We use enumerate to apply the subsampling rate for precision and plotting
        for k, (t_k, Y_k) in enumerate(zip(time_steps, Y_measurements)):

            # --- NON-CAUSALITY: Use ALL measurements (Past, Present, and Future) ---
            # The 'if t_k > t_target: continue' from the causal version is REMOVED.
            # --------------------------

            # Delta_t will be positive for past/present (forecast) and negative for future (backcast)
            Delta_t = t_target - t_k

            # 1. Propagate Measurement (Forecast/Backcast to t_target)
            # The expm(A * Delta_t) handles both positive and negative Delta_t
            Phi = expm(A * Delta_t)
            Y_mapped = Phi @ Y_k

            # Decide whether to use this point for precision/plotting based on subsampling
            if k % SMOOTHING_SUBSAMPLE_RATE == 0:
                mapped_cloud.append(Y_mapped) # Only add subsampled points to the cloud

                # 2. Calculate V_C(Delta_t) and P_C(Delta_t)
                # The function handles the covariance for both forward and backward propagation
                V_C = compute_propagated_covariance(Delta_t)
                try:
                    # Add precision to total accumulator: P_total += V_C_inv
                    P_total += np.linalg.inv(V_C)
                except np.linalg.LinAlgError:
                    print(f"Warning: Singular V_C encountered at t_k={t_k:.2f}")

        mapped_cloud = np.array(mapped_cloud)

        # 3. Calculate Smoothed Covariance V_smoother = P_total_inv
        try:
            V_smoother = np.linalg.inv(P_total)
        except np.linalg.LinAlgError:
            print(f"Error: Total precision P_total is singular at t_target={t_target}")
            continue # Skip plotting this ellipse

        # Plot the cloud (only subsampled points are included now)
        ax.scatter(mapped_cloud[:, 0], mapped_cloud[:, 1],
                   color=color, marker='.', s=20, alpha=0.5,
                   label=f'Mapped Cloud ($t = {t_target}$)', zorder=2)

        # --- Plot Analytic Smoother Ellipse ---
        # The smoother covariance (V_smoother) is the result of combining ALL measurement information.
        plot_ellipse(ax, X_target, V_smoother, 'black',
                     label_suffix=r'Analytic $\mathbf{V}_{\text{smoother}}$' if not smoother_legend_added else None,
                     linestyle='-', linewidth=3, sigma=2) # sigma=2 (2-sigma ellipse)

        # Only add the ellipse label once for the legend
        smoother_legend_added = True

    # 3. Mark Start Point
    ax.plot(X_true[0, 0], X_true[0, 1], 'o', color='green', markersize=8,
            label='Start (t=0)', zorder=3)

    # 4. Plot Formatting
    # ax.set_title(r'Optimal Non-Causal Smoother Covariance $\mathbf{V}_{\text{smoother}}$ from All Measurements', fontsize=16)
    ax.set_xlabel(r'State $x_1$', fontsize=12)
    ax.set_ylabel(r'State $x_2$', fontsize=12)
    ax.set_aspect('equal', adjustable='box')
    plt.xlim(-7,9)
    plt.ylim(-8,8)
    ax.grid(True, linestyle=':', alpha=0.5)

    # Handle legend with duplicate labels
    handles, labels = ax.get_legend_handles_labels()
    unique = []
    seen = set()
    for h, l in zip(handles, labels):
        if l not in seen:
            unique.append((h, l))
            seen.add(l)
    # ax.legend(*zip(*unique), loc='lower right')

    plt.tight_layout()
    plt.show()


# --- Run Simulation and Plot ---
print("--- Running SDE Non-Causal Smoother Plot ---")
time_steps, X_true, Y_measurements = simulate_sde(FIXED_END_TIME, DT)

# The plot function is now modified for non-causal smoothing
plot_smoothed_measurements(time_steps, X_true, Y_measurements, TARGET_TIMES, TARGET_COLORS)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm, eig
from matplotlib.collections import LineCollection
from numpy.linalg import cholesky # Needed for SDE simulation noise generation

# --- Helper function for drawing confidence ellipses (copied from previous version) ---
def plot_ellipse(ax, mean, cov, color, label_suffix, sigma=10, linestyle='-', linewidth=2): # *** MODIFIED: sigma=3 ***
    """
    Plots the confidence ellipse for a given covariance matrix.
    """
    # Calculate eigenvalues and eigenvectors
    # D: eigenvalues (variances), V: eigenvectors (principal axes)
    D, V = np.linalg.eig(cov)

    # Ensure real, non-negative eigenvalues for the 2D case
    D = np.maximum(np.real(D), 1e-9)

    # Calculate the angle of rotation for the largest eigenvector
    angle = np.degrees(np.arctan2(V[1, 0], V[0, 0]))

    # Calculate width and height of the ellipse (sigma * 2 * sqrt(eigenvalue))
    width, height = sigma * 2 * np.sqrt(D)

    # Create the ellipse object
    from matplotlib.patches import Ellipse
    ellipse = Ellipse(xy=mean, width=width, height=height, angle=angle,
                      edgecolor=color, facecolor='none', linestyle=linestyle, linewidth=linewidth, zorder=4)

    # Add the ellipse to the axis
    ax.add_patch(ellipse)

    # Add a dummy artist for the legend entry if this is the first one
    if label_suffix:
        ax.plot([], [], color=color, linestyle=linestyle, linewidth=linewidth,
                label=label_suffix)


# --- Explicit Eigendecomposition Provided by User ---
# Eigenvalues (Lambda)
Lambda_provided = np.array([
    [-0.1 - 1j, 0],
    [0, -0.1 + 1j]
], dtype=complex)

# Eigenvectors (S)
S_provided = np.array([
    [1 - 1j, 1 + 1j],
    [1, 1]
], dtype=complex)

# Inverse of Eigenvectors (S_inv)
S_inv_provided = 0.5 * np.array([
    [1j, 1 - 1j],
    [-1j, 1 + 1j]
], dtype=complex)

# --- System Parameters ---
# Continuous-Time System Matrix (A = S @ Lambda @ S_inv)
A = np.real(S_provided @ Lambda_provided @ S_inv_provided)
# Note: This results in A = [[-0.1, -1.0], [1.0, -0.1]], a circular spiral.

# Process Noise Std Dev (sigma_process) from simulation
SIGMA_PROCESS = 0.2
# Correlation between x1 and x2 process noise components
CORRELATION_Q = 0.4
# Continuous-Time Process Noise Covariance (Q = G*G^T).
Q_variance = SIGMA_PROCESS**2
Q = np.array([
    [Q_variance, CORRELATION_Q * Q_variance],
    [CORRELATION_Q * Q_variance, Q_variance]
])
# Cholesky decomposition for process noise (L*L^T = Q)
L = cholesky(Q)

# Measurement Matrix (C = H in simulation)
C = np.identity(2)

# Measurement Noise Std Dev (sigma_measurement) from simulation
SIGMA_MEASUREMENT = 0.5
# Correlation between x1 and x2 measurement noise components
CORRELATION_R = 0.3
# Measurement Noise Covariance (R) - Now non-diagonal
R_variance = SIGMA_MEASUREMENT**2
R = np.array([
    [R_variance, CORRELATION_R * R_variance],
    [CORRELATION_R * R_variance, R_variance]
])
# Cholesky decomposition for measurement noise (M*M^T = R)
M = cholesky(R)

# --- Visualization Parameters ---
FIXED_END_TIME = 15.0 # Total simulation time (T)
INITIAL_STATE = np.array([5.0, 0.0]) # Starting point of the spiral at t=0

# *** MODIFIED: Decrease DT for more measurements (3000 total) ***
DT = 0.005 # Time step for Euler-Maruyama simulation

N_POINTS_MEASUREMENT = int(FIXED_END_TIME / DT) # Total number of measurements

# --- New Smoothing Visualization Parameters ---
TARGET_TIMES = [1.0, 5.0, 10.0, 13.0, 15.0] # The 5 fixed times to map all measurements to
TARGET_COLORS = ['green', 'blue', 'red', 'orange', 'purple'] # Colors for the 5 resulting clouds

# *** MODIFIED: Decrease subsample rate for more points in plot and precision sum ***
SMOOTHING_SUBSAMPLE_RATE = 5 # Use 1 out of every 5 measurements for smoothing and plotting (600 points)

# --- Analytical DLE Solution (Kept for completeness) ---

# Pre-calculate Eigenbasis components for faster DLE solution
Lambda = np.diag(Lambda_provided)
S = S_provided
S_inv = S_inv_provided
S_dagger = S.conj().T
S_inv_dagger = S_inv.conj().T
Lambda_Q = np.diag(S_inv @ Q @ S_inv_dagger)
Lambda_C = np.diag(S_inv @ C @ S)
Lambda_R = np.diag(S_inv @ R @ S_inv_dagger)


def scalar_dle_solution_phi(lambda_k, lambda_Q_k, dt):
    """
    Computes the scalar solution phi(lambda, lambda_Q, dt) for the DLE.
    This function handles both positive (forecast) and negative (backcast) dt.
    """
    real_lambda = np.real(lambda_k)

    if np.isclose(real_lambda, 0.0):
        # Case for pure imaginary or zero eigenvalues (unstable/constant)
        return lambda_Q_k * dt
    else:
        # Standard stable/unstable case
        exponential_term = np.exp(2 * real_lambda * dt)
        denominator = -2 * real_lambda
        return lambda_Q_k * (1 - exponential_term) / denominator


def compute_propagated_covariance(dt):
    """
    Computes the total propagated measurement covariance V_C(dt).
    This function is used for both positive (forecast) and negative (backcast) dt.
    """
    d = A.shape[0]
    Lambda_V_diag = np.zeros(d, dtype=complex)
    for k in range(d):
        Lambda_V_diag[k] = scalar_dle_solution_phi(Lambda[k], Lambda_Q[k], dt)

    abs_Lambda_C_sq = np.abs(Lambda_C)**2
    term1 = abs_Lambda_C_sq * Lambda_V_diag
    real_Lambda = np.real(Lambda)
    exp_term = np.exp(2 * real_Lambda * dt)
    term2 = exp_term * Lambda_R
    Lambda_V_C_diag = term1 + term2
    V_C = S @ np.diag(Lambda_V_C_diag) @ S_dagger
    return np.real(V_C)


# --- Simulation Function (Updated to return data) ---

def simulate_sde(T, dt):
    """
    Simulates the true state (X) and noisy measurements (Y) of the 2D SDE.
    Returns the time steps, true states, and measurements.
    """
    N_steps = int(T / dt)
    time_steps = np.arange(0, T, dt)

    X = np.zeros((N_steps, 2)) # True state
    Y = np.zeros((N_steps, 2)) # Measured state

    X[0] = INITIAL_STATE

    # Pre-calculate factor for process noise term
    L_sqrt_dt = L @ (np.sqrt(dt) * np.identity(2))

    # Euler-Maruyama Simulation Loop
    for k in range(N_steps - 1):
        # 1. Process Noise (dW_t ~ N(0, dt*Q))
        Z_k = np.random.randn(2)
        dW_term = L_sqrt_dt @ Z_k

        # 2. True State Update: X_k+1 = X_k + A * X_k * dt + dW_term
        drift_term = (A @ X[k]) * dt
        X[k+1] = X[k] + drift_term + dW_term

        # 3. Measurement Noise (v_t ~ N(0, R))
        V_k = M @ np.random.randn(2)

        # 4. Measured State: Y_k = C @ X_k + v_t
        Y[k] = C @ X[k] + V_k

    # Final measurement at the last time step
    Y[-1] = C @ X[-1] + M @ np.random.randn(2)

    return time_steps, X, Y

# --- Smoother (Non-Causal Filter) Mapping and Plotting Function ---

def plot_smoothed_measurements(time_steps, X_true, Y_measurements, target_times, target_colors):
    """
    Maps ALL measurements (Y_k for all t_k) to the specified target times (t_target)
    using the matrix exponential (Forecast for past, Backcast for future),
    and overlays the optimal NON-CAUSAL SMOOTHER covariance.
    """
    fig, ax = plt.subplots(figsize=(10, 10))

    # 1. Plot the True State Trajectory
    ax.plot(X_true[:, 0], X_true[:, 1], color='black', linestyle='-', linewidth=1.5, alpha=0.9,
            label='True Trajectory', zorder=1)

    # Flag to ensure the smoother ellipse legend is only added once
    smoother_legend_added = False

    # 2. Map and Plot Measurement Clouds and Optimal Smoother Ellipses
    for t_target, color in zip(target_times, target_colors):

        # Find the true state at the target time for plotting the center
        target_index = np.argmin(np.abs(time_steps - t_target))
        X_target = X_true[target_index]

        # Initialize total precision (P_total = sum(P_C) for all k)
        P_total = np.zeros((2, 2))
        mapped_cloud = []

        # --- Map Measurements (Forecast/Backcast) & Accumulate Precision ---
        for k, (t_k, Y_k) in enumerate(zip(time_steps, Y_measurements)):

            Delta_t = t_target - t_k

            # 1. Propagate Measurement (Forecast/Backcast to t_target)
            Phi = expm(A * Delta_t)
            Y_mapped = Phi @ Y_k

            # Decide whether to use this point for precision/plotting based on subsampling
            if k % SMOOTHING_SUBSAMPLE_RATE == 0:
                mapped_cloud.append(Y_mapped) # Only add subsampled points to the cloud

                # 2. Calculate V_C(Delta_t) and P_C(Delta_t)
                V_C = compute_propagated_covariance(Delta_t)
                try:
                    # Add precision to total accumulator: P_total += V_C_inv
                    P_total += np.linalg.inv(V_C)
                except np.linalg.LinAlgError:
                    print(f"Warning: Singular V_C encountered at t_k={t_k:.2f}")

        mapped_cloud = np.array(mapped_cloud)

        # 3. Calculate Smoothed Covariance V_smoother = P_total_inv
        try:
            V_smoother = np.linalg.inv(P_total)
        except np.linalg.LinAlgError:
            print(f"Error: Total precision P_total is singular at t_target={t_target}")
            continue # Skip plotting this ellipse

        # Plot the cloud (only subsampled points are included now)
        ax.scatter(mapped_cloud[:, 0], mapped_cloud[:, 1],
                   color=color, marker='.', s=10, alpha=0.3, # Decreased marker size/alpha for density
                   label=f'Mapped Cloud ($t = {t_target}$)', zorder=2)

        # --- Plot Analytic Smoother Ellipse ---
        # The 'plot_ellipse' function is called here, using the updated 'sigma' from its definition
        plot_ellipse(ax, X_target, V_smoother, 'black',
                     label_suffix=r'Analytic $\mathbf{V}_{\text{smoother}}$' if not smoother_legend_added else None,
                     linestyle='-', linewidth=3) # sigma=3 (3-sigma ellipse) is now default

        # Only add the ellipse label once for the legend
        smoother_legend_added = True

    # 3. Mark Start Point
    ax.plot(X_true[0, 0], X_true[0, 1], 'o', color='green', markersize=8,
            label='Start (t=0)', zorder=3)

    # 4. Plot Formatting
    # ax.set_title(r'Optimal Non-Causal Smoother Covariance with Larger Ellipses', fontsize=16)
    ax.set_xlabel(r'State $x_1$', fontsize=12)
    ax.set_ylabel(r'State $x_2$', fontsize=12)
    ax.set_aspect('equal', adjustable='box')
    plt.xlim(-7,9)
    plt.ylim(-8,8)
    ax.grid(True, linestyle=':', alpha=0.5)

    # Handle legend with duplicate labels
    handles, labels = ax.get_legend_handles_labels()
    unique = []
    seen = set()
    for h, l in zip(handles, labels):
        if l not in seen:
            unique.append((h, l))
            seen.add(l)
    # ax.legend(*zip(*unique), loc='lower right')

    plt.tight_layout()
    plt.show()


# --- Run Simulation and Plot ---
print(f"--- Running SDE Non-Causal Smoother Plot with {N_POINTS_MEASUREMENT} Total Measurements (DT={DT}), Sigma={3} ---")
time_steps, X_true, Y_measurements = simulate_sde(FIXED_END_TIME, DT)

# The plot function is now modified for non-causal smoothing
plot_smoothed_measurements(time_steps, X_true, Y_measurements, TARGET_TIMES, TARGET_COLORS)

Fix the measurement time $t_j$ and vary the target time $t_i$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm, eig
from matplotlib.collections import LineCollection

# --- Explicit Eigendecomposition Provided by User ---
# Eigenvalues (Lambda)
Lambda_provided = np.array([
    [-0.1 - 1j, 0],
    [0, -0.1 + 1j]
], dtype=complex)

# Eigenvectors (S)
S_provided = np.array([
    [1 - 1j, 1 + 1j],
    [1, 1]
], dtype=complex)

# Inverse of Eigenvectors (S_inv)
S_inv_provided = 0.5 * np.array([
    [1j, 1 - 1j],
    [-1j, 1 + 1j]
], dtype=complex)

# --- System Parameters ---
# Continuous-Time System Matrix (A = S @ Lambda @ S_inv)
A = np.real(S_provided @ Lambda_provided @ S_inv_provided)
# Note: This results in A = [[-0.1, -1.0], [1.0, -0.1]], a circular spiral.

# Process Noise Std Dev (sigma_process) from simulation
SIGMA_PROCESS = 0.2
# Correlation between x1 and x2 process noise components
CORRELATION_Q = 0.4
# Continuous-Time Process Noise Covariance (Q = G*G^T).
Q_variance = SIGMA_PROCESS**2
Q = np.array([
    [Q_variance, CORRELATION_Q * Q_variance],
    [CORRELATION_Q * Q_variance, Q_variance]
])

# Measurement Matrix (C = H in simulation)
C = np.identity(2)

# Measurement Noise Std Dev (sigma_measurement) from simulation
SIGMA_MEASUREMENT = 0.5
# Correlation between x1 and x2 measurement noise components
CORRELATION_R = 0.3
# Measurement Noise Covariance (R) - Now non-diagonal
R_variance = SIGMA_MEASUREMENT**2
R = np.array([
    [R_variance, CORRELATION_R * R_variance],
    [CORRELATION_R * R_variance, R_variance]
])

# --- Visualization Parameters ---
FIXED_END_TIME = 15.0 # Total time for propagation (t_i runs from 0 to this value)
N_POINTS = 500        # Number of points to draw the spiral (higher for smooth width)
MIN_LINE_WIDTH = 1.0  # Reduced min width to increase visual dynamic range
MAX_LINE_WIDTH = 20.0 # Increased max width to increase visual dynamic range
INITIAL_STATE = np.array([5.0, 0.0]) # Starting point of the spiral at t=0

# --- Pre-calculate Eigenbasis components for faster DLE solution ---
# Use the explicit matrices provided by the user for the decoupling
Lambda = np.diag(Lambda_provided)
S = S_provided
S_inv = S_inv_provided

# Calculate remaining basis components based on the provided S and S_inv
S_dagger = S.conj().T
S_inv_dagger = S_inv.conj().T

# Recalculate Lambda_Q, Lambda_C, and Lambda_R using the new eigenbasis and correlated Q/R
Lambda_Q = np.diag(S_inv @ Q @ S_inv_dagger)
Lambda_C = np.diag(S_inv @ C @ S)
Lambda_R = np.diag(S_inv @ R @ S_inv_dagger)


def scalar_dle_solution_phi(lambda_k, lambda_Q_k, dt):
    """
    Computes the scalar solution phi(lambda, lambda_Q, dt) for the DLE.
    This corresponds to the k-th diagonal element of Lambda_V(dt).
    """
    real_lambda = np.real(lambda_k)

    if np.isclose(real_lambda, 0.0):
        # Case Re(lambda) = 0 (Purely imaginary eigenvalue)
        return lambda_Q_k * dt
    else:
        # Case Re(lambda) != 0 (Stable or unstable system)
        exponential_term = np.exp(2 * real_lambda * dt)
        denominator = -2 * real_lambda
        return lambda_Q_k * (1 - exponential_term) / denominator


def compute_propagated_covariance(dt):
    """
    Computes the total propagated measurement covariance V_C(dt) using the
    pre-calculated eigenbasis components.
    """
    d = A.shape[0]

    # 3. Compute Lambda_V(dt) (Decoupled State Covariance)
    Lambda_V_diag = np.zeros(d, dtype=complex)
    for k in range(d):
        # Lambda[k] is the k-th eigenvalue, Lambda_Q[k] is the k-th element of Lambda_Q
        Lambda_V_diag[k] = scalar_dle_solution_phi(Lambda[k], Lambda_Q[k], dt)

    # 4. Compute Lambda_V^C(dt) (Decoupled Total Covariance)
    # Lambda_V^C = |Lambda_C|^2 * Lambda_V + exp(2 * Re(Lambda) * dt) * Lambda_R
    abs_Lambda_C_sq = np.abs(Lambda_C)**2

    term1 = abs_Lambda_C_sq * Lambda_V_diag

    real_Lambda = np.real(Lambda)
    exp_term = np.exp(2 * real_Lambda * dt)
    term2 = exp_term * Lambda_R

    Lambda_V_C_diag = term1 + term2

    # 5. Compute V_C(dt) (Total Propagated Covariance)
    V_C = S @ np.diag(Lambda_V_C_diag) @ S_dagger

    # Ensure V_C is real
    return np.real(V_C)


def plot_spiral_uncertainty(fixed_t_i, n_points, min_width, max_width):
    """
    Plots the deterministic spiral trajectory from t=0 to t=fixed_t_i.
    The linewidth at each point (t_i) represents the uncertainty of the state
    at t_i, propagated from the fixed anchor at t_j=0, where Delta_t = t_i.
    """
    # t_current corresponds to t_i, varying from 1e-6 to fixed_t_i
    t_current_values = np.linspace(1e-6, fixed_t_i, n_points)
    trajectory = []
    v_c_amplitudes = [] # Stores sqrt(Trace(V_C))

    # 1. Calculate trajectory and V_C(Delta_t) for all current points (t_i)
    for t_i in t_current_values:
        # Mean Path: x(t_i) = exp(A*t_i) * x(0)
        # This defines the location of the line segment being drawn.
        state_at_ti = expm(A * t_i) @ INITIAL_STATE
        trajectory.append(state_at_ti)

        # Calculate time difference: Delta_t = t_i - t_j = t_i - 0
        Delta_t = t_i

        # Covariance V_C(Delta_t) is the uncertainty of the state at t_i,
        # propagated from the measurement at t_j=0.
        V_C = compute_propagated_covariance(Delta_t)

        # Scalar amplitude measure: sqrt of the trace (sum of variances)
        amplitude = np.sqrt(np.trace(V_C))
        v_c_amplitudes.append(amplitude)

    trajectory = np.array(trajectory)
    v_c_amplitudes = np.array(v_c_amplitudes)

    # 2. Normalize amplitude to LineWidth range [MIN_LINE_WIDTH, MAX_LINE_WIDTH]
    min_amp = np.min(v_c_amplitudes)
    max_amp = np.max(v_c_amplitudes)

    if max_amp - min_amp < 1e-9:
        linewidths = np.ones_like(v_c_amplitudes) * min_width
    else:
        # Scale amplitude normalized to [0, 1]
        normalized_amp = (v_c_amplitudes - min_amp) / (max_amp - min_amp)
        # Scale to linewidths between min_width and max_width
        linewidths = min_width + normalized_amp * (max_width - min_width)

    # 3. Prepare segments for LineCollection
    points = trajectory.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    # 4. Create LineCollection
    fig, ax = plt.subplots(figsize=(10, 10))

    # Static light blue color for monochrome uncertainty envelope
    MONOCHROME_COLOR = 'skyblue'

    lc = LineCollection(segments,
                        linewidths=linewidths,
                        # Using 'color' for static color across all segments
                        color=MONOCHROME_COLOR,
                        alpha=0.8,
                        capstyle='round',
                        label=r'Propagated Uncertainty $\propto \sqrt{\mathrm{Tr}(\mathbf{V}^C(t_i))}$')
    ax.add_collection(lc)

    # 5. The thin black line representing the core mean trajectory.
    ax.plot(trajectory[:, 0], trajectory[:, 1], color='black', linewidth=1,
            label=r'Mean Trajectory (Current Time $t_i$)', zorder=2)

    # Plot the fixed start state t_j=0
    start_state = expm(A * 0) @ INITIAL_STATE
    ax.plot(start_state[0], start_state[1], 'o', color='green', markersize=8,
            label=r'Fixed Anchor $\mathbf{x}(t_j=0)$', zorder=5)

    # 7. Plot formatting
    # ax.set_title(r'Uncertainty Propagation $\mathbf{V}^C(\Delta t)$ from Fixed Anchor $t_j=0$', fontsize=16)
    ax.set_xlabel(r'State $x$', fontsize=12)
    ax.set_ylabel(r'State $y$', fontsize=12)
    # ax.set_xlabel(r'$x$', fontsize=12)
    # ax.set_ylabel(r'$y$', fontsize=12)
    ax.set_aspect('equal', adjustable='box')
    ax.grid(True, linestyle=':', alpha=0.5)
    plt.xlim(-5,7)
    plt.ylim(-5,7)
    # ax.legend(loc='lower right')
    plt.tight_layout()
    plt.show()


# --- Run Calculation and Plot ---
print("--- Running SDE Covariance Propagation Plot ---")
print(f"Propagating from fixed anchor t_j=0 to current time t_i up to {FIXED_END_TIME} s.")

plot_spiral_uncertainty(FIXED_END_TIME, N_POINTS, MIN_LINE_WIDTH, MAX_LINE_WIDTH)

Fix the target time $t_i$ and vary the measurement time $t_j$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm, eig
from matplotlib.collections import LineCollection

# --- Explicit Eigendecomposition Provided by User ---
# Eigenvalues (Lambda)
Lambda_provided = np.array([
    [-0.1 - 1j, 0],
    [0, -0.1 + 1j]
], dtype=complex)

# Eigenvectors (S)
S_provided = np.array([
    [1 - 1j, 1 + 1j],
    [1, 1]
], dtype=complex)

# Inverse of Eigenvectors (S_inv)
S_inv_provided = 0.5 * np.array([
    [1j, 1 - 1j],
    [-1j, 1 + 1j]
], dtype=complex)

# --- System Parameters ---
# Continuous-Time System Matrix (A = S @ Lambda @ S_inv)
A = np.real(S_provided @ Lambda_provided @ S_inv_provided)
# Note: This results in A = [[-0.1, -1.0], [1.0, -0.1]], a circular spiral.

# Process Noise Std Dev (sigma_process) from simulation
SIGMA_PROCESS = 0.1
# Correlation between x1 and x2 process noise components
CORRELATION_Q = 0.5
# Continuous-Time Process Noise Covariance (Q = G*G^T).
Q_variance = SIGMA_PROCESS**2
Q = np.array([
    [Q_variance, CORRELATION_Q * Q_variance],
    [CORRELATION_Q * Q_variance, Q_variance]
])

# Measurement Matrix (C = H in simulation)
C = np.identity(2)

# Measurement Noise Std Dev (sigma_measurement) from simulation
SIGMA_MEASUREMENT = 0.5
# Correlation between x1 and x2 measurement noise components
CORRELATION_R = 0.3
# Measurement Noise Covariance (R) - Now non-diagonal
R_variance = SIGMA_MEASUREMENT**2
R = np.array([
    [R_variance, CORRELATION_R * R_variance],
    [CORRELATION_R * R_variance, R_variance]
])

# --- Visualization Parameters ---
FIXED_END_TIME = 15.0 # Fixed current time (t_i) for the forecast target
N_POINTS = 500        # Number of points to draw the spiral (higher for smooth width)
MIN_LINE_WIDTH = 1.0  # Reduced min width to increase visual dynamic range
MAX_LINE_WIDTH = 20.0 # Increased max width to increase visual dynamic range
INITIAL_STATE = np.array([5.0, 0.0]) # Starting point of the spiral at t=0

# --- Pre-calculate Eigenbasis components for faster DLE solution ---
# Use the explicit matrices provided by the user for the decoupling
Lambda = np.diag(Lambda_provided)
S = S_provided
S_inv = S_inv_provided

# Calculate remaining basis components based on the provided S and S_inv
S_dagger = S.conj().T
S_inv_dagger = S_inv.conj().T

# Recalculate Lambda_Q, Lambda_C, and Lambda_R using the new eigenbasis and correlated Q/R
Lambda_Q = np.diag(S_inv @ Q @ S_inv_dagger)
Lambda_C = np.diag(S_inv @ C @ S)
Lambda_R = np.diag(S_inv @ R @ S_inv_dagger)


def scalar_dle_solution_phi(lambda_k, lambda_Q_k, dt):
    """
    Computes the scalar solution phi(lambda, lambda_Q, dt) for the DLE.
    This corresponds to the k-th diagonal element of Lambda_V(dt).
    """
    real_lambda = np.real(lambda_k)

    if np.isclose(real_lambda, 0.0):
        # Case Re(lambda) = 0 (Purely imaginary eigenvalue)
        return lambda_Q_k * dt
    else:
        # Case Re(lambda) != 0 (Stable or unstable system)
        exponential_term = np.exp(2 * real_lambda * dt)
        denominator = -2 * real_lambda
        return lambda_Q_k * (1 - exponential_term) / denominator


def compute_propagated_covariance(dt):
    """
    Computes the total propagated measurement covariance V_C(dt) using the
    pre-calculated eigenbasis components.
    """
    d = A.shape[0]

    # 3. Compute Lambda_V(dt) (Decoupled State Covariance)
    Lambda_V_diag = np.zeros(d, dtype=complex)
    for k in range(d):
        # Lambda[k] is the k-th eigenvalue, Lambda_Q[k] is the k-th element of Lambda_Q
        Lambda_V_diag[k] = scalar_dle_solution_phi(Lambda[k], Lambda_Q[k], dt)

    # 4. Compute Lambda_V^C(dt) (Decoupled Total Covariance)
    # Lambda_V^C = |Lambda_C|^2 * Lambda_V + exp(2 * Re(Lambda) * dt) * Lambda_R
    abs_Lambda_C_sq = np.abs(Lambda_C)**2

    term1 = abs_Lambda_C_sq * Lambda_V_diag

    real_Lambda = np.real(Lambda)
    exp_term = np.exp(2 * real_Lambda * dt)
    term2 = exp_term * Lambda_R

    Lambda_V_C_diag = term1 + term2

    # 5. Compute V_C(dt) (Total Propagated Covariance)
    V_C = S @ np.diag(Lambda_V_C_diag) @ S_dagger

    # Ensure V_C is real
    return np.real(V_C)


def plot_spiral_uncertainty(fixed_t_i, n_points, min_width, max_width):
    """
    Plots the deterministic spiral trajectory from t=0 to t=fixed_t_i.
    The linewidth at each point (t_j) represents the uncertainty of the forecast
    from t_j to the fixed target time t_i, where Delta_t = t_i - t_j.
    """
    # t_anchor corresponds to t_j, varying from 1e-6 to fixed_t_i
    t_anchor_values = np.linspace(1e-6, fixed_t_i, n_points)
    trajectory = []
    v_c_amplitudes = [] # Stores sqrt(Trace(V_C))

    # 1. Calculate trajectory and V_C(Delta_t) for all anchor points (t_j)
    for t_j in t_anchor_values:
        # Mean Path: x(t_j) = exp(A*t_j) * x(0)
        state_at_tj = expm(A * t_j) @ INITIAL_STATE
        trajectory.append(state_at_tj)

        # Calculate time difference: Delta_t = t_i - t_j
        Delta_t = fixed_t_i - t_j

        # Covariance V_C(Delta_t) is the uncertainty of the forecast from t_j to t_i
        V_C = compute_propagated_covariance(Delta_t)

        # Scalar amplitude measure: sqrt of the trace (sum of variances)
        amplitude = np.sqrt(np.trace(V_C))
        v_c_amplitudes.append(amplitude)

    trajectory = np.array(trajectory)
    v_c_amplitudes = np.array(v_c_amplitudes)

    # 2. Normalize amplitude to LineWidth range [MIN_LINE_WIDTH, MAX_LINE_WIDTH]
    min_amp = np.min(v_c_amplitudes)
    max_amp = np.max(v_c_amplitudes)

    if max_amp - min_amp < 1e-9:
        linewidths = np.ones_like(v_c_amplitudes) * min_width
    else:
        # Scale amplitude normalized to [0, 1]
        normalized_amp = (v_c_amplitudes - min_amp) / (max_amp - min_amp)
        # Scale to linewidths between min_width and max_width
        linewidths = min_width + normalized_amp * (max_width - min_width)

    # 3. Prepare segments for LineCollection
    points = trajectory.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    # 4. Create LineCollection
    fig, ax = plt.subplots(figsize=(10, 10))

    # Static light blue color for monochrome uncertainty envelope
    MONOCHROME_COLOR = 'skyblue'

    lc = LineCollection(segments,
                        linewidths=linewidths,
                        # Using 'color' for static color across all segments
                        color=MONOCHROME_COLOR,
                        alpha=0.8,
                        capstyle='round',
                        label=r'Forecast Uncertainty $\propto \sqrt{\mathrm{Tr}(\mathbf{V}^C(t_i - t_j))}$')
    ax.add_collection(lc)

    # 5. The thin black line representing the core mean trajectory.
    ax.plot(trajectory[:, 0], trajectory[:, 1], color='black', linewidth=1,
            label=r'Mean Trajectory (Anchor $t_j$)', zorder=2)

    # Plot the fixed end state t_i
    end_state = expm(A * fixed_t_i) @ INITIAL_STATE
    ax.plot(end_state[0], end_state[1], 'D', color='red', markersize=8,
            label=r'Fixed Target State $\mathbf{x}(t_i)$', zorder=5)

    # 7. Plot formatting
    ax.set_title(r'Forecast Uncertainty $\mathbf{V}^C(\Delta t)$ from Anchor $t_j$ to Fixed Target $t_i$', fontsize=16)
    ax.set_xlabel(r'State $x_1$', fontsize=12)
    ax.set_ylabel(r'State $x_2$', fontsize=12)
    ax.set_aspect('equal', adjustable='box')
    ax.grid(True, linestyle=':', alpha=0.5)
    ax.legend(loc='upper right')
    plt.tight_layout()
    plt.show()


# --- Run Calculation and Plot ---
print("--- Running SDE Covariance Forecast Plot ---")
print(f"Forecasting to fixed time t_i = {FIXED_END_TIME} s from past anchors t_j.")

plot_spiral_uncertainty(FIXED_END_TIME, N_POINTS, MIN_LINE_WIDTH, MAX_LINE_WIDTH)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm, eig
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# --- Explicit Eigendecomposition Provided by User ---
# Eigenvalues (Lambda)
Lambda_provided = np.array([
    [-0.1 - 1j, 0],
    [0, -0.1 + 1j]
], dtype=complex)

# Eigenvectors (S)
S_provided = np.array([
    [1 - 1j, 1 + 1j],
    [1, 1]
], dtype=complex)

# Inverse of Eigenvectors (S_inv)
S_inv_provided = 0.5 * np.array([
    [1j, 1 - 1j],
    [-1j, 1 + 1j]
], dtype=complex)

# --- System Parameters ---
# Continuous-Time System Matrix (A = S @ Lambda @ S_inv)
# We calculate A explicitly from the provided matrices and take the real part,
# as A must be real for a real-valued SDE.
A = np.real(S_provided @ Lambda_provided @ S_inv_provided)
# Resulting A is: [[-0.1, -1.0], [1.0, -0.1]] (a circular spiral)

# Process Noise Std Dev (sigma_process) from simulation
SIGMA_PROCESS = 0.1
# Correlation between x1 and x2 process noise components
CORRELATION_Q = 0.5
# Continuous-Time Process Noise Covariance (Q = G*G^T).
Q_variance = SIGMA_PROCESS**2
Q = np.array([
    [Q_variance, CORRELATION_Q * Q_variance],
    [CORRELATION_Q * Q_variance, Q_variance]
])

# Measurement Matrix (C = H in simulation)
C = np.identity(2)

# Measurement Noise Std Dev (sigma_measurement) from simulation
SIGMA_MEASUREMENT = 0.5
# Correlation between x1 and x2 measurement noise components
CORRELATION_R = 0.3
# Measurement Noise Covariance (R) - Now non-diagonal
R_variance = SIGMA_MEASUREMENT**2
R = np.array([
    [R_variance, CORRELATION_R * R_variance],
    [CORRELATION_R * R_variance, R_variance]
])

# --- Visualization Parameters ---
FIXED_END_TIME = 10.0 # Fixed current time (t_i) for the forecast target
N_POINTS_TRAJECTORY = 500 # For drawing the smooth mean trajectory
N_ELLIPSES = 10           # Number of discrete points for drawing ellipses
INITIAL_STATE = np.array([5.0, 0.0]) # Starting point of the spiral at t=0
SIGMA_LEVEL = 2.0         # Confidence level for ellipse (e.g., 2.0 for 2-sigma)

# --- Pre-calculate Eigenbasis components for faster DLE solution ---
# Use the explicit matrices provided by the user for the decoupling
Lambda = np.diag(Lambda_provided)
S = S_provided
S_inv = S_inv_provided

# Calculate remaining basis components based on the provided S and S_inv
S_dagger = S.conj().T
S_inv_dagger = S_inv.conj().T

# Recalculate Lambda_Q, Lambda_C, and Lambda_R using the new eigenbasis
Lambda_Q = np.diag(S_inv @ Q @ S_inv_dagger)
Lambda_C = np.diag(S_inv @ C @ S) # Note: C is identity here, so this step might be redundant, but maintained for generality
Lambda_R = np.diag(S_inv @ R @ S_inv_dagger)


def scalar_dle_solution_phi(lambda_k, lambda_Q_k, dt):
    """
    Computes the scalar solution phi(lambda, lambda_Q, dt) for the DLE.
    This corresponds to the k-th diagonal element of Lambda_V(dt).
    """
    real_lambda = np.real(lambda_k)

    if np.isclose(real_lambda, 0.0):
        return lambda_Q_k * dt
    else:
        exponential_term = np.exp(2 * real_lambda * dt)
        denominator = -2 * real_lambda
        return lambda_Q_k * (1 - exponential_term) / denominator


def compute_propagated_covariance(dt):
    """
    Computes the total propagated measurement covariance V_C(dt) using the
    pre-calculated eigenbasis components.
    """
    d = A.shape[0]

    # 3. Compute Lambda_V(dt) (Decoupled State Covariance)
    Lambda_V_diag = np.zeros(d, dtype=complex)
    for k in range(d):
        # Lambda[k] is the k-th eigenvalue, Lambda_Q[k] is the k-th element of Lambda_Q
        Lambda_V_diag[k] = scalar_dle_solution_phi(Lambda[k], Lambda_Q[k], dt)

    # 4. Compute Lambda_V^C(dt) (Decoupled Total Covariance)
    # Lambda_V^C = |Lambda_C|^2 * Lambda_V + exp(2 * Re(Lambda) * dt) * Lambda_R
    abs_Lambda_C_sq = np.abs(Lambda_C)**2

    term1 = abs_Lambda_C_sq * Lambda_V_diag

    real_Lambda = np.real(Lambda)
    exp_term = np.exp(2 * real_Lambda * dt)
    term2 = exp_term * Lambda_R

    Lambda_V_C_diag = term1 + term2

    # 5. Compute V_C(dt) (Total Propagated Covariance)
    V_C = S @ np.diag(Lambda_V_C_diag) @ S_dagger

    # Ensure V_C is real (since A, Q, C, R are real, V_C must be real)
    return np.real(V_C)


def plot_ellipse(ax, mean, cov, sigma_level, color, label=''):
    """
    Draws a 2D confidence ellipse on the given axes.
    """
    # Decompose covariance matrix
    eigenvalues, eigenvectors = np.linalg.eig(cov)

    # We must ensure we use the real parts and correct ordering for plotting
    eigenvalues = np.real(eigenvalues)

    # Ensure non-negative eigenvalues (for covariance matrix)
    eigenvalues[eigenvalues < 0] = 1e-9

    # Determine the rotation angle (angle of the principal eigenvector)
    # The angle corresponds to the first eigenvector
    angle = np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0])
    rotation_deg = np.degrees(angle)

    # Determine the ellipse dimensions (2*sigma_level * sqrt(eigenvalue))
    # Using the two principal axes
    width = sigma_level * 2 * np.sqrt(eigenvalues[0])
    height = sigma_level * 2 * np.sqrt(eigenvalues[1])

    # Create the Ellipse patch
    ellipse = Ellipse(
        xy=mean,
        width=width,
        height=height,
        angle=rotation_deg,
        edgecolor=color,
        facecolor='none',
        linestyle='--',
        alpha=0.7,
        linewidth=1.5,
        label=label if label else None
    )

    # Add to plot
    ax.add_patch(ellipse)


def plot_spiral_uncertainty_ellipses(fixed_t_i, n_ellipses):
    """
    Plots the deterministic spiral trajectory and overlays uncertainty ellipses
    at discrete anchor points t_j.
    """

    # 1. Define time points for smooth trajectory (for the black line)
    time_trajectory = np.linspace(0, fixed_t_i, N_POINTS_TRAJECTORY)
    trajectory_points = np.array([expm(A * t) @ INITIAL_STATE for t in time_trajectory])

    # 2. Define discrete anchor points for ellipses (t_j)
    # Ensure t_j ranges from 0 (long forecast) to just before t_i (short forecast)
    t_anchor_values = np.linspace(0.01, fixed_t_i - 0.01, n_ellipses)

    # Setup Plot
    fig, ax = plt.subplots(figsize=(10, 10))

    # Draw the smooth mean trajectory line
    ax.plot(trajectory_points[:, 0], trajectory_points[:, 1], color='black', linewidth=1,
            label=r'Mean Trajectory $\mathbf{x}(t_j)$', zorder=2)

    # Calculate and plot ellipses
    for i, t_j in enumerate(t_anchor_values):
        # Position of the ellipse center (Mean trajectory at t_j)
        mean_position = expm(A * t_j) @ INITIAL_STATE

        # Time difference for forecast: Delta_t = t_i - t_j
        Delta_t = fixed_t_i - t_j

        # Calculate Covariance V_C(Delta_t)
        V_C = compute_propagated_covariance(Delta_t)

        # Use a single color (light blue) for monochrome
        ellipse_color = 'skyblue'

        # Only label the first ellipse for the legend
        label = r'Forecast $\mathbf{V}^C(\Delta t)$ Ellipse' if i == 0 else ''

        # Plot the ellipse at the mean position, representing the uncertainty of the
        # forecast from t_j to t_i.
        plot_ellipse(ax, mean_position, V_C, SIGMA_LEVEL, ellipse_color, label=label)

        # Optional: Mark the anchor points (t_j)
        ax.plot(mean_position[0], mean_position[1], 'o', color='gray', markersize=3, zorder=3)

    # Plot the fixed end state t_i (Target)
    end_state = expm(A * fixed_t_i) @ INITIAL_STATE
    ax.plot(end_state[0], end_state[1], 'D', color='red', markersize=8,
            label=r'Fixed Target State $\mathbf{x}(t_i)$', zorder=5)

    # Plot formatting
    ax.set_title(r'Forecast Uncertainty Ellipses ($2\sigma$) from Anchor $t_j$ to Fixed Target $t_i$', fontsize=16)
    ax.set_xlabel(r'State $x_1$', fontsize=12)
    ax.set_ylabel(r'State $x_2$', fontsize=12)
    ax.set_aspect('equal', adjustable='box')
    ax.grid(True, linestyle=':', alpha=0.5)
    ax.legend(loc='upper right')
    plt.tight_layout()
    plt.show()


# --- Run Calculation and Plot ---
print("--- Running SDE Covariance Ellipse Forecast Plot ---")
print(f"Targeting fixed time t_i = {FIXED_END_TIME} s from {N_ELLIPSES} past anchors t_j.")

plot_spiral_uncertainty_ellipses(FIXED_END_TIME, N_ELLIPSES)

In [ ]:
# import numpy as np

# # Define the complex-valued matrices as specified:
# # Lambda (diagonal matrix of eigenvalues)
# Lambda = np.array([
#     [-0.1 - 1j, 0],
#     [0, -0.1 + 1j]
# ], dtype=complex)

# # S (matrix of eigenvectors)
# S = np.array([
#     [1 - 1j, 1 + 1j],
#     [1, 1]
# ], dtype=complex)

# # S_inv (inverse matrix S^-1)
# S_inv = 0.5 * np.array([
#     [1j, 1 - 1j],
#     [-1j, 1 + 1j]
# ], dtype=complex)

# # Verify S @ S_inv is close to Identity (optional but good practice)
# # print("S @ S_inv verification (should be close to Identity):\n", S @ S_inv)

# # Compute A = S @ Lambda @ S_inv
# A = S @ Lambda @ S_inv

# # Display the result, rounding to a small number of decimal places
# # to show that the resulting matrix A is real, as expected for
# # a real-valued SDE system matrix.
# print("Matrix Lambda (Eigenvalues):\n", Lambda)
# print("\nMatrix S (Eigenvectors):\n", S)
# print("\nMatrix S_inv (Inverse Eigenvectors):\n", S_inv)
# print("\nComputed Matrix A (S @ Lambda @ S_inv):\n")
# # Print rounded A, explicitly converting very small imaginary parts to zero for clarity
# print(np.round(A.real, 8) + 0j * np.round(A.imag, 15))